In [ ]:
import czmodel

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 MB 2.6 MB/s eta 0:00:00


## Create a CZModel from the trained Keras model

In this section we export the trained model to the CZModel format using the czmodel library and some additional meta data all possible parameter choices are described in the [ANN model specification](https://pypi.org/project/czmodel/) that can be found on the PyPi packager for `czmodel`.

### Define Meta-Data

We first define the meta-data needed to run the model within the Intellesis infrastructure. The `czmodel` package offers a named tuple `ModelMetadata` that allows to either parse as JSON file as described or to directly specify the parameters as shown below.

### Create a Model Specification Object

The export functions provided by the `czmodel` package expect a `ModelSpec` tuple that features the Keras model to be exported and the corresponding model metadata.

Therefore, we wrap our model and the `model_metadata` instance into a `ModelSpec` object.

In [1]:
modelpath = r"C:\Users\m1srh\Documents\GitHub\RMS_Hackathon_Napari\napari-czmodel-segment\src\napari_czmodel_segment\_data\Nucleus Segmentation Grayscale v3\f8d00820-f6b3-429b-8bc1-bcca612227de"



In [5]:
# Function provided by the PyPI package called czmodel (by ZEISS)
from czmodel.model_metadata import ModelMetadata, ModelSpec, ModelType
from czmodel.legacy_model_metadata import ModelMetadata as LegacyModelMetadata, ModelSpec as LegacyModelSpec
from czmodel import DefaultConverter, LegacyConverter
from czmodel.util.transforms import Shift, Scale

# Define dimensions - ZEN Intellesis requires fully defined spatial dimensions in the meta data of the CZANN model.
# The ZEN TilingClient uses the input shape in the meta data to infer the tile size to pass an image to the inferencer.
# Important: The tile size has to be chosen s.t. inference is possible with the minimum hardware requirements of Intellesis
# Optional: Define target spatial dimensions of the model for inference.
input_size = 1024

# Define the model metadata
model_metadata = ModelMetadata(
    input_shape=[input_size, input_size, 1],
    output_shape=[input_size, input_size, 3],
    model_type=ModelType.SINGLE_CLASS_SEMANTIC_SEGMENTATION,
    classes=["Background", "Nucleus", "Border"],
    model_name="Simple_Nuclei_SegmentationModel ONNX",
    min_overlap=[90, 90],
)
model_spec = ModelSpec(
    model=modelpath,
    model_metadata=model_metadata,
    license_file=None
)

### Perform model export into *.czann / *.czseg file format

The converters from the `czmodel` library offers two functions to perform the actual export. 

* `convert_from_json_spec` allows to provide a JSON file containing all the information of a ModelSpec object and converts a model in SavedModel format on disk to a `.czann` / `.czseg` file that can be loaded with ZEN.
* `convert_from_model_spec` expects a `ModelSpec` object, an output path and name and optionally target spatial dimensions for the expected input of the exported model. From this information it creates a `.czann` / `.czseg` file containing the specified model.

Currently, `czmodel` offers two converters:
* DefaultConverter: Converts a model to a *.czann file.
* LegacyConverter: Converts a model to a *.czseg file (legacy format).

In [6]:
DefaultConverter().convert_from_model_spec(
    model_spec=model_spec, 
    output_path=r"c:\Users\m1srh\Documents\GitHub\RMS_Hackathon_Napari\napari-czmodel-segment\src\napari_czmodel_segment\_data", 
    output_name='simple_nuclei_segmodel',
    spatial_dims=(input_size, input_size),
    #preprocessing=preprocessing
)

# In the example above there will be a ""./czmodel_output/simple_nuclei_segmodel.czann" file saved on disk.

INFO:tensorflow:Assets written to: C:\Users\m1srh\AppData\Local\Temp\tmpojbadh6h\assets


'--tag' not specified for saved_model. Using --tag serve
'--signature_def' not specified, using first signature: serving_default


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


## Remarks
The generated .czann and .czseg files can be directly loaded into ZEN Intellesis to perform segmentation tasks with the trained model.
If there is already a trained model in SavedModel format present on disk, it can also be converted by providing the path to the saved model directory instead of a Keras `Model` object. The `czmodel` library will implicitly load the model from the provided path.

The `czmodel` library also provides a `convert_from_json_spec` function that accepts a JSON file with the above mentioned meta data behind the key `ModelMetadata` which will implicitly be deserialized into a `ModelMetadata` object, the model path and optionally a license file:
```json
{
    "ModelMetadata": {
        "Type": "SingleClassSemanticSegmentation",
        "Classes": ["Background", "Nucleus"],
        "InputShape": [1024, 1024, 1],
        "OutputShape": [1024, 1024, 2],
        "ModelName": "Nuclei Segmentation Model From JSON",
        "MinOverlap": [128, 128]
    },
    "ModelPath": "./saved_tf2_model_output/",
    "LicenseFile": null
}
```

This information can be copied to a file e.g. in the current working directory `./model_conversion_spec.json` that also contains the trained model in SavedModel format e.g. generated by the following line:

In [ ]:
# save the trained TF2.SavedModel as a folder structure
# The folder + the JSON file can be also used to import the model in ZEN

model.save(os.path.join(os.path.join(base_directory, modelfolder), './saved_tf2_model_output/'))

In [ ]:
tf.keras.backend.clear_session()